In [3]:
import numpy as np

class HiddenMarkovModel:
    def __init__(self, states, observations,
                 start_prob, trans_prob, emit_prob):
        self.states = states
        self.observations = observations
        self.start_prob = start_prob
        self.trans_prob = trans_prob
        self.emit_prob = emit_prob

    def forward(self, obs_seq):
        n_states = len(self.states)
        T = len(obs_seq)

        fwd = np.zeros((n_states, T))

        # Initialization
        for s in range(n_states):
            fwd[s, 0] = self.start_prob[s] * self.emit_prob[s, self.observations.index(obs_seq[0])]
       
        # Recursion
        for t in range(1, T):
            for s in range(n_states):
                fwd[s, t] = sum(fwd[s_prev, t-1] * self.trans_prob[s_prev, s] for s_prev in range(n_states)) \
                            * self.emit_prob[s, self.observations.index(obs_seq[t])]
       
        # Termination
        prob = sum(fwd[s, T-1] for s in range(n_states))
        return prob

    def viterbi(self, obs_seq):
        n_states = len(self.states)
        T = len(obs_seq)

        v = np.zeros((n_states, T))
        path = np.zeros((n_states, T), dtype=int)

        # Initialization
        for s in range(n_states):
            v[s, 0] = self.start_prob[s] * self.emit_prob[s, self.observations.index(obs_seq[0])]
            path[s, 0] = 0

        # Recursion
        for t in range(1, T):
            for s in range(n_states):
                (prob, state) = max(
                    (v[s_prev, t-1] * self.trans_prob[s_prev, s], s_prev)
                    for s_prev in range(n_states)
                )
                v[s, t] = prob * self.emit_prob[s, self.observations.index(obs_seq[t])]
                path[s, t] = state

        # Termination
        (prob, state) = max((v[s, T-1], s) for s in range(n_states))

        # Backtrack
        best_path = [state]
        for t in range(T-1, 0, -1):
            state = path[state, t]
            best_path.insert(0, state)

        state_sequence = [self.states[i] for i in best_path]
        return (prob, state_sequence)

# Example usage
states = ['Rainy', 'Sunny']
observations = ['walk', 'shop', 'clean']
start_prob = [0.6, 0.4]
trans_prob = np.array([
    [0.7, 0.3],
    [0.4, 0.6]
])
emit_prob = np.array([
    [0.1, 0.4, 0.5],
    [0.6, 0.3, 0.1]
])

hmm = HiddenMarkovModel(states, observations, start_prob, trans_prob, emit_prob)

obs_sequence = ['shop', 'walk', 'clean']
print("Forward Algorithm (probability of the observed sequence):")
print(hmm.forward(obs_sequence))

print("\nViterbi Algorithm (most likely hidden state path):")
prob, path = hmm.viterbi(obs_sequence)
print(f"Probability: {prob}")
print(f"Path: {path}")

Forward Algorithm (probability of the observed sequence):
0.030672

Viterbi Algorithm (most likely hidden state path):
Probability: 0.00864
Path: ['Sunny', 'Sunny', 'Rainy']
